# State Farm Distracted Driver Detection


[State Farm Distracted Driver Detection](https://www.kaggle.com/c/state-farm-distracted-driver-detection#evaluation)

## Action Plan
### 1. Data Preparation and Preprocessing
### 2. Finetune and Train Model
### 3. Generate and Validate Predictions 
### 4. Submit predictions to Kaggle

## 2. Finetune and Train Model

In [1]:
%cd "~/kaggle/state-farm-driver-detection/code"
%pwd

/home/ubuntu/kaggle/state-farm-driver-detection/code


'/home/ubuntu/kaggle/state-farm-driver-detection/code'

In [2]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
CODE_HOME_DIR = current_dir
DATA_HOME_DIR = CODE_HOME_DIR + '/../input/'
print(CODE_HOME_DIR)
print(DATA_HOME_DIR)

/home/ubuntu/kaggle/state-farm-driver-detection/code
/home/ubuntu/kaggle/state-farm-driver-detection/code/../input/


In [4]:
#import modules
#from importlib import reload

import utils
from utils import *

import vgg16bn_ted
from vgg16bn_ted import Vgg16BN; 

%matplotlib inline

## Setup 

#### Setup Paths

In [5]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/' # + '/sample/' 
test_path = path + '/test/'
results_path = path + '/results/'
train_path = path + '/train/'
valid_path = path + '/valid/'
model_path = path + '/models/'
if not os.path.exists(model_path): os.mkdir(model_path)

/home/ubuntu/kaggle/state-farm-driver-detection/input


## First training

In [7]:
vgg = Vgg16BN(size=(256, 256))
#vgg = Vgg16BN()
model = vgg.model

In [8]:
batch_size=64

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 3, 256, 256)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 64, 256, 256)      1792      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 64, 256, 256)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 128, 128)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 128, 64, 64)       0         
__________

In [11]:
#Finetune the model
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, shuffle=False, batch_size=batch_size)
#vgg.finetune(batches)

Found 17940 images belonging to 10 classes.
Found 4484 images belonging to 10 classes.


In [ ]:
vgg.fit(batches, val_batches, epochs=2)